# **14.6 Context_Managers_and_With_Statement**

The `with` statement is one of Python's most elegant features — it automatically handles setup and cleanup for resources like files, database connections, and locks. You've been using it with files already (`with open(...) as f:`), but in this lesson you'll understand *how* it works, why it matters, and how to create your own context managers for Pokemon game resources like battle sessions, save file locks, and connection pools.

---

## **The Problem: Manual Cleanup**

Without `with`, you must manually close resources — and if an exception occurs between opening and closing, the resource leaks. This is error-prone and verbose.

In [ ]:
# BAD: Manual file handling (prone to leaks)
file = open('pokemon_team.txt', 'w')
file.write('Pikachu,Electric,25\n')
# If an error occurs here, close() never runs!
file.close()

# BETTER: Try/finally ensures cleanup
file = open('pokemon_team.txt', 'w')
try:
    file.write('Charizard,Fire,36\n')
    # Even if an exception occurs, finally always runs
finally:
    file.close()

# BEST: with statement handles it automatically
with open('pokemon_team.txt', 'w') as file:
    file.write('Blastoise,Water,36\n')
    # File auto-closes when block ends — even if an exception occurs

print("with statement is cleanest and safest!")

---

## **How with Works: __enter__ and __exit__**

Any object that defines `__enter__()` and `__exit__()` methods is a **context manager**. When you use `with`, Python calls `__enter__()` at the start and `__exit__()` at the end — guaranteed.

In [ ]:
# Demonstrating the with statement protocol
class ShowProtocol:
    """A simple context manager that shows when methods are called."""
    
    def __enter__(self):
        print("1. __enter__ called (setup happens here)")
        return "Resource object"  # This becomes the 'as' variable
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("3. __exit__ called (cleanup happens here)")
        # Return False to propagate exceptions, True to suppress them
        return False

# Using the context manager
print("Before with block")
with ShowProtocol() as resource:
    print(f"2. Inside with block, resource = {resource!r}")
print("4. After with block\n")

# Execution order:
# 1. __enter__ is called
# 2. Code in the with block runs
# 3. __exit__ is called (even if an exception occurred)
# 4. Control returns to code after the with block

---

## **Creating a Simple Context Manager**

Let's create a context manager for Pokemon battles that tracks when battles start and end, ensuring cleanup even if the battle crashes.

In [ ]:
import time

class BattleSession:
    """
    Context manager for Pokemon battles.
    Tracks battle start/end and ensures cleanup.
    """
    
    def __init__(self, attacker: str, defender: str):
        self.attacker = attacker
        self.defender = defender
        self.start_time = None
    
    def __enter__(self):
        """Called when entering the 'with' block."""
        self.start_time = time.time()
        print(f"⚔ Battle started: {self.attacker} vs {self.defender}")
        return self  # Return self so 'as battle' gives access to the instance
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Called when exiting the 'with' block."""
        elapsed = time.time() - self.start_time
        
        if exc_type is None:
            # Normal exit — no exception
            print(f"✓ Battle completed in {elapsed:.2f} seconds")
        else:
            # Exception occurred
            print(f"✗ Battle interrupted by {exc_type.__name__}: {exc_val}")
            print(f"  Duration: {elapsed:.2f} seconds")
        
        # Return False to let exceptions propagate
        return False

# Use the battle context manager
with BattleSession("Pikachu", "Onix") as battle:
    print(f"  {battle.attacker} used Thunderbolt!")
    time.sleep(0.5)  # Simulate battle time
    print(f"  {battle.defender} fainted!")

print()

# Even if an exception occurs, cleanup still happens
try:
    with BattleSession("Charizard", "Blastoise") as battle:
        print(f"  {battle.attacker} used Flamethrower!")
        raise RuntimeError("Battle bug!")
        print("This won't print")
except RuntimeError:
    print("  Exception caught outside with block")

---

## **Using contextlib for Simple Context Managers**

Writing full classes with `__enter__` and `__exit__` is verbose. Python's `contextlib` module provides a decorator that turns generator functions into context managers.

In [ ]:
from contextlib import contextmanager
import time

@contextmanager
def battle_timer(pokemon_name: str):
    """
    Context manager that times how long a Pokemon's turn takes.
    """
    print(f"{pokemon_name}'s turn starts...")
    start = time.time()
    
    try:
        yield  # Code in the 'with' block runs here
    finally:
        # Cleanup always happens here
        elapsed = time.time() - start
        print(f"{pokemon_name}'s turn took {elapsed:.2f} seconds")

# Use the context manager
with battle_timer("Pikachu"):
    print("  Choosing move...")
    time.sleep(0.3)
    print("  Pikachu used Thunderbolt!")
    time.sleep(0.2)

print()

# Context manager that yields a value
@contextmanager
def catch_attempt(pokemon_name: str, catch_rate: float):
    """Context manager for Pokemon catch attempts."""
    import random
    
    print(f"Throwing Pokéball at {pokemon_name}...")
    success = random.random() < catch_rate
    
    try:
        yield success  # This value becomes the 'as' variable
    finally:
        if success:
            print(f"✓ {pokemon_name} was caught!")
        else:
            print(f"✗ {pokemon_name} broke free!")

with catch_attempt("Mewtwo", catch_rate=0.03) as caught:
    if caught:
        print("  Adding to Pokédex...")
    else:
        print("  Trying again...")

---

## **Multiple Context Managers**

You can use multiple `with` statements in a single line or nest them. Each context manager's cleanup happens in reverse order (last in, first out).

In [ ]:
from pathlib import Path

# Multiple files in one with statement (Python 3.1+)
source = Path('source.txt')
dest = Path('destination.txt')

source.write_text('Pikachu,Electric,25\nCharizard,Fire,36\n')

# Both files are opened and both are auto-closed
with source.open('r') as src, dest.open('w') as dst:
    for line in src:
        dst.write(line.upper())  # Copy but uppercase

print("Files copied:")
print(dest.read_text())

# Nested context managers
@contextmanager
def log_section(name: str):
    print(f"{'='*40}")
    print(f"Starting: {name}")
    print(f"{'='*40}")
    try:
        yield
    finally:
        print(f"{'='*40}")
        print(f"Finished: {name}")
        print(f"{'='*40}\n")

with log_section("Pokemon Battle"):
    with battle_timer("Pikachu"):
        print("  Thunderbolt used!")
        time.sleep(0.1)

---

## **Suppressing Exceptions**

If `__exit__` returns `True`, the exception is suppressed (swallowed). This is useful for optional cleanup where you don't want failures to crash the program.

In [ ]:
class SuppressErrors:
    """Context manager that catches and logs exceptions without propagating."""
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            print(f"Caught {exc_type.__name__}: {exc_val}")
            return True  # Suppress the exception
        return False

# Use it
print("Before with block")

with SuppressErrors():
    print("Inside with block")
    raise ValueError("Something went wrong!")
    print("This won't print")  # Skipped due to exception

print("After with block — still running!\n")

# Python's built-in contextlib.suppress does this
from contextlib import suppress

with suppress(FileNotFoundError):
    # If file doesn't exist, no exception is raised
    Path('nonexistent.txt').unlink()

print("File deletion attempted — no error even if missing")

---

## **Practical: Save File Lock Manager**

A real-world example: prevent two processes from writing to the same save file simultaneously by using a lock file.

In [ ]:
from pathlib import Path
from contextlib import contextmanager
import time

class SaveFileLocked(Exception):
    """Raised when save file is already locked by another process."""

@contextmanager
def save_file_lock(save_path: Path, timeout: float = 5.0):
    """
    Context manager that creates a lock file to prevent concurrent writes.
    Automatically removes lock on exit.
    """
    lock_file = save_path.with_suffix('.lock')
    start_time = time.time()
    
    # Wait for lock to become available
    while lock_file.exists():
        if time.time() - start_time > timeout:
            raise SaveFileLocked(f"Save file locked: {save_path}")
        time.sleep(0.1)
    
    # Acquire lock
    lock_file.write_text(f"Locked at {time.time()}")
    print(f"🔒 Lock acquired: {lock_file.name}")
    
    try:
        yield save_path  # Caller can now safely write to save_path
    finally:
        # Release lock — always happens
        if lock_file.exists():
            lock_file.unlink()
            print(f"🔓 Lock released: {lock_file.name}")

# Use the lock manager
save_file = Path('game_save.json')

with save_file_lock(save_file) as locked_file:
    print(f"Writing to {locked_file}...")
    locked_file.write_text('{"trainer": "Ash", "badges": 8}')
    time.sleep(0.5)  # Simulate slow write
    print("Write complete")

print("\nLock automatically released — safe for next write")

---

## **Practice Exercises**

### **Task 1: Basic with Statement**

Use `with` to write a Pokemon name to a file.

**Expected Output:**
```
File written
```

In [ ]:
# Your code here:


### **Task 2: Create Simple Context Manager**

Create a class with `__enter__` and `__exit__` that prints messages.

**Expected Output:**
```
Entering
Inside
Exiting
```

In [ ]:
# Your code here:


### **Task 3: Use @contextmanager**

Create a context manager using `@contextmanager` that prints start/end.

**Expected Output:**
```
Battle start
Fighting...
Battle end
```

In [ ]:
# Your code here:


### **Task 4: Yield a Value**

Create a context manager that yields a Pokemon dict.

**Expected Output:**
```
Pikachu
```

In [ ]:
# Your code here:


### **Task 5: Timer Context Manager**

Create a context manager that times the code block.

**Expected Output:**
```
Code took 0.50 seconds
```

In [ ]:
# Your code here:


### **Task 6: Handle Exception in __exit__**

Create a context manager that catches exceptions in `__exit__`.

**Expected Output:**
```
Caught ValueError: test error
```

In [ ]:
# Your code here:


### **Task 7: Multiple with Statements**

Use two context managers in one `with` line.

**Expected Output:**
```
First enter
Second enter
Inside both
Second exit
First exit
```

In [ ]:
# Your code here:


### **Task 8: contextlib.suppress**

Use `contextlib.suppress` to ignore a `FileNotFoundError`.

**Expected Output:**
```
No error raised
```

In [ ]:
# Your code here:


### **Task 9: Cleanup Guaranteed**

Show that `__exit__` runs even if an exception occurs.

**Expected Output:**
```
Enter
Exit (cleanup ran)
```

In [ ]:
# Your code here:


### **Task 10: Battle Session Manager**

Create a context manager for Pokemon battles that logs start/end and handles errors.

**Expected Output:**
```
Battle: Pikachu vs Onix
Battle complete
```

In [ ]:
# Your code here:


---

## **Summary**

- `with` statement guarantees cleanup — even if exceptions occur
- Context managers implement `__enter__` and `__exit__`
- `__enter__` runs at the start (setup)
- `__exit__` runs at the end (cleanup) — always
- `@contextmanager` decorator creates context managers from generators
- `yield` in the middle — code before = setup, code after = cleanup
- Multiple `with` statements: `with a, b:` or nest them
- `__exit__` returning `True` suppresses exceptions
- `contextlib.suppress(Exception)` — built-in exception suppressor
- Common uses: files, locks, timers, database connections, temporary state

---

## **Quick Reference**

```python
# Basic with statement
with open('file.txt', 'r') as f:
    data = f.read()
# File auto-closes

# Custom context manager (class)
class MyContext:
    def __enter__(self):
        print("Setup")
        return self  # Value for 'as'
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Cleanup")
        return False  # Propagate exceptions

# Custom context manager (generator)
from contextlib import contextmanager

@contextmanager
def my_context():
    print("Setup")
    try:
        yield "value"  # Code block runs here
    finally:
        print("Cleanup")  # Always runs

# Multiple contexts
with open('a.txt') as f1, open('b.txt') as f2:
    ...

# Suppress exceptions
from contextlib import suppress
with suppress(FileNotFoundError):
    Path('file.txt').unlink()
```